### Step 1: Load preprocessed data

In [ ]:
from datasets import Dataset, DatasetDict

# Step 1: Load the Parquet files
train_dataset = Dataset.from_parquet(['data/german_0.parquet', 'data/german_1.parquet'])
test_dataset = Dataset.from_parquet('data/german_2.parquet')

# Step 2: transform audio from list to array
train_dataset = train_dataset.with_format("np", columns=["audio"], output_all_columns=True)
test_dataset = test_dataset.with_format("np", columns=["audio"], output_all_columns=True)

# Step 3: combine to DatasetDict

dataset_ger = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

### Step 2: Preprocessing - tokenization of training corpus

In [ ]:
from transformers import WhisperTokenizer, WhisperFeatureExtractor, WhisperProcessor
import torch

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v3")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v3", language="german", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3", language="german", task="transcribe")

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["labels"]).input_ids
    return batch

dataset_ger = dataset_ger.map(prepare_dataset, remove_columns=dataset_ger.column_names["train"], num_proc= torch.set_num_threads(1))

### Step 3: initialize model

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")
model.generation_config.language = "german"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")   # converted feature (log-Mel) to PyTorch tensors

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore padded label for correct loss
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor,
                                                     decoder_start_token_id=model.config.decoder_start_token_id)



### Step 4: Define Evaluation 

In [ ]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


### Step 5: define training environment

In [5]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./training_weights/whisper-large-v3-ger",  
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=5e-6,
    warmup_steps=500,
    max_steps=6000,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_ger["train"],
    eval_dataset=dataset_ger["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


### Step 6: run model

In [ ]:
trainer.train()

### Step 7: save fine-tuned model

In [ ]:
# After training or fine-tuning the model:
trainer.save_model('./whisper-finetuned-model')
tokenizer.save_pretrained('./whisper-finetuned-model')